<a href="https://colab.research.google.com/github/meryemka112/Data-Mining-Labs/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install pyspark

In [30]:
!pip install mmh3


In [31]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,823 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,422 kB]
Fetched 12.3 MB in 3s (3,510 kB/s)
Reading package lists... Don

In [32]:
# # Créer une session Spark
# spark = SparkSession.builder.appName("Exemple").getOrCreate()

# # Charger un fichier CSV
# df = spark.read.csv("metaverse_transactions_dataset.csv", header=True, inferSchema=True)

# # Afficher le schéma et une partie des données
# df.printSchema()
# df.show()



In [33]:
# type(df)

In [34]:
# df.select(['timestamp','amount']).show()

In [35]:
#spark.stop()

In [36]:
import pyspark
from pyspark.sql import DataFrame, SparkSession
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col
import hashlib
from pyspark.sql.functions import lower, regexp_replace, split, udf
from pyspark.sql.types import ArrayType, StringType, LongType, IntegerType

Initialiser Apache Spark

In [37]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Document Similarity") \
    .getOrCreate()
sc = spark.sparkContext


In [38]:
import mmh3

**Classe Shingling**

In [53]:
# class Shingling():
#     def __init__(self, k=10):
#         """
#         k: length of shingles (in words)
#         """
#         self.k = k

#     def clean_text(self, df: DataFrame, text_col: str) -> DataFrame:
#         # Mettre en minuscule, supprimer caractères spéciaux, réduire espaces multiples
#         df = df.withColumn(text_col, lower(col(text_col)))
#         df = df.withColumn(text_col, regexp_replace(col(text_col), r"[^a-z0-9\s]", ""))
#         df = df.withColumn(text_col, regexp_replace(col(text_col), r"\s+", " "))
#         return df

#     def tokenize(self, df: DataFrame, text_col: str) -> DataFrame:
#         df = df.withColumn("tokens", split(col(text_col), "\\s+"))

#         def clean_tokens(tokens):
#             # Supprimer None et chaînes vides
#             return [str(t).strip() for t in tokens if t and str(t).strip()]

#         clean_udf = udf(clean_tokens, ArrayType(StringType())) #udf(...) transforme clean_tokens en fonction distribuée Spark.#Spark ne peut pas directement appliquer des fcts Python, besoin d'un wrapper -> udf
#         df = df.withColumn("tokens", clean_udf(col("tokens")))
#         return df

#     def generate_shingles(self, df: DataFrame) -> DataFrame:
#         k = self.k

#         def shingle_function(tokens):
#             tokens = [str(t).strip() for t in tokens if t and str(t).strip()]
#             if len(tokens) < k:
#                 return []
#             shingles = [" ".join(tokens[i:i+k]) for i in range(len(tokens) - k + 1)]
#             # Supprimer shingles vides ou None
#             return [s for s in shingles if s and s.strip()]

#         shingle_udf = udf(shingle_function, ArrayType(StringType()))
#         df = df.withColumn("shingles", shingle_udf(col("tokens")))
#         return df

#     def hash_shingles(self, df: DataFrame) -> DataFrame:
#         def hash_list(shingles):
#             hashed = []
#             for s in shingles:
#                 if s and s.strip():
#                     h = mmh3.hash64(s.strip(), signed=False)[0] #retourne entier non signé entre 0 et 2^64 -1
#                     h = h % (2**63)  # forcer des entier entre 0 et 2^63 -1 #Sinon j'ai eu un overflow pour LongType et donc des valeurs NULL de hashage (mon prob initial)
#                     #h = mmh3.hash("some string", signed=False)  # 32-bit unsigned integer
#                     hashed.append(h)
#             return hashed

#         hash_udf = udf(hash_list, ArrayType(LongType()))   #longtype entre - 2^63 et 2^63 -1
#         df = df.withColumn("hashed_shingles", hash_udf(col("shingles")))
#         return df

#     def transform(self, df: DataFrame, text_col: str) -> DataFrame:
#         df = self.clean_text(df, text_col)
#         df = self.tokenize(df, text_col)
#         df = self.generate_shingles(df)
#         df = self.hash_shingles(df)
#         return df

class Shingling():
    def __init__(self, k=10):
        self.k = k

    def transform(self, df: DataFrame, text_col: str) -> DataFrame:
        from pyspark.sql.functions import col, lower, regexp_replace, split, udf
        from pyspark.sql.types import ArrayType, LongType, StringType
        import mmh3

        # Nettoyage du texte
        df = df.withColumn(text_col, lower(col(text_col)))
        df = df.withColumn(text_col, regexp_replace(col(text_col), r"[^a-z0-9\s]", ""))
        df = df.withColumn(text_col, regexp_replace(col(text_col), r"\s+", " "))

        # Tokenization
        df = df.withColumn("tokens", split(col(text_col), "\\s+"))

        # Génération des shingles
        def shingles_fn(tokens):
            k = self.k
            tokens = [str(t).strip() for t in tokens if t and str(t).strip()]
            if len(tokens) < k:
                return []
            return [" ".join(tokens[i:i+k]) for i in range(len(tokens) - k + 1)]

        shingle_udf = udf(shingles_fn, ArrayType(StringType()))
        df = df.withColumn("shingles", shingle_udf(col("tokens")))

        # Hash des shingles
        def hash_fn(shingles):
            hashed = []
            for s in shingles:
                if s and s.strip():
                    h = mmh3.hash64(s.strip(), signed=False)[0]
                    h = h % (2**63)
                    hashed.append(h)
            return hashed

        hash_udf = udf(hash_fn, ArrayType(LongType()))
        df = df.withColumn("hashed_shingles", hash_udf(col("shingles")))

        # Garder seulement filename et hashed_shingles
        df = df.select("filename", "hashed_shingles")
        return df


In [40]:
data = [
    (1, "Spark is a distributed computing engine for large-scale data processing."),
    (2, "Hadoop is a framework for distributed computing and storage.")
]

df = spark.createDataFrame(data, ["id", "text"])

shingler = Shingling(k=3)
df_result = shingler.transform(df, "text")

df_result.select("id", "shingles", "hashed_shingles").show(truncate=False)


+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id |shingles                                                                                                                                                                           |hashed_shingles                                                                                                                                                         |
+---+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------

In [41]:
#Version python bof
# import hashlib

# class Shingling:
#     def __init__(self, k=10):
#         """
#         k: length of shingles (in words)
#         """
#         self.k = k

#     def tokenize(self, text):
#         """
#         Simple tokenization: split text into words
#         """
#         return text.lower().split()

#     def generate_shingles(self, tokens):
#         """
#         Generate k-word shingles from token list
#         """
#         shingles = []
#         for i in range(len(tokens) - self.k + 1):
#             shingle = ' '.join(tokens[i:i+self.k])
#             shingles.append(shingle)
#         return shingles

#     def hash_shingle(self, shingle):
#         """
#         Deterministic hash using MD5 truncated to 64 bits
#         """
#         h = int(hashlib.md5(shingle.encode('utf-8')).hexdigest()[:16], 16)
#         h = h % (2**63)  # ensure fits in 64-bit signed integer
#         return h

#     def shingle_document(self, text):
#         """
#         Complete pipeline: tokenize → generate shingles → hash shingles
#         Returns: ordered list of hashed shingles
#         """
#         tokens = self.tokenize(text)
#         shingles = self.generate_shingles(tokens)
#         hashed_shingles = [self.hash_shingle(s) for s in shingles]
#         return hashed_shingles


**Classe CompareSets**

In [42]:
class CompareSets():
  #méthode statique ici car n'accéde pas à l'attribut de l'objet, n'a besoin que de 2 arguments set1 et set 2
  @staticmethod
  def calculate_jaccard_similarity(set1,set2):
    if not set1 or not set2:
      return 0.0
    s1 = set(set1)
    s2 = set(set2)
    intersection = s1.intersection(s2)
    union = s1.union(s2)
    jaccard_similarity = len(intersection) / len(union)
    return jaccard_similarity


In [43]:
class MinHashing():
  def __init__(self,n):
    self.n = n
  # def hash_set(self,shingle_set):
  #   n = self.n
  #   min_i = []
  #   for i in range(n):
  #     hashed = []
  #     for x in shingle_set:
  #       h = mmh3.hash(str(x), seed=i, signed=False)
  #       hashed.append(h)
  #     min_i.append(min(hashed))
  #   return min_i
  def minhash_udf(self):
        def _minhash(shingle_list):
          if not shingle_list:
            return [0]*self.n
          return [min(mmh3.hash(str(x), seed=i, signed=False) for x in shingle_list)
                      for i in range(self.n)]
        return udf(_minhash, ArrayType(LongType()))






In [44]:
# shingles_doc1 = [6840193950337781150, 1695454882927409067, 7334696573069238721]
# minhasher = MinHashing(n=5)
# signature = minhasher.hash_set(shingles_doc1)

# print(signature)
# # Output example: [122343123, 2332231, 99221231, 28381223, 112392391]


In [45]:
class CompareSignatures():
  @staticmethod
  def compare(sig1,sig2):
    if len(sig1) != len(sig2):
      raise ValueError("Signatures must have the same length")

    matches = sum(1 for a,b in zip(sig1,sig2) if a == b)
    return matches /len(sig1)


In [46]:
# uploaded_files = ["1.txt", "2.txt", "3.txt","4.txt","5.txt"]

# data = []
# for filename in uploaded_files:
#     with open(filename, "r", encoding="utf-8") as f:
#         text = f.read()
#         data.append((filename, text))

# df = spark.createDataFrame(data, ["filename", "text"])

# # 3️⃣ Appliquer la classe Shingling
# shingling = Shingling(k=5)
# df_shingles = shingling.transform(df, text_col="text")

# # 4️⃣ Voir le résultat
# df_shingles.select("filename", "shingles", "hashed_shingles").show(truncate=50)

In [47]:
#Préparer les docs
import os
from itertools import combinations

# Liste des fichiers uploadés
uploaded_files = ["1.txt", "2.txt", "3.txt", "4.txt", "5.txt"]

data = []
for filename in uploaded_files:
    with open(filename, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
        data.append((filename, text))

# Créer un DataFrame Spark
df_docs = spark.createDataFrame(data, ["filename", "text"])


In [54]:
shingler = Shingling(k=10)
df_shingles = shingler.transform(df_docs, "text")
df_shingles.show(truncate=False)


+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [55]:
n_hashes = 50 # nombre de fonctions de hachage pour le minhash
minhasher = MinHashing(n_hashes)

# Ajouter une colonne "minhash" au DataFrame
df_shingles = df_shingles.withColumn("minhash", minhasher.minhash_udf()(col("hashed_shingles")))
df_shingles.show(truncate=False)


+--------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [56]:
threshold_minhash = 0.6
rows = df_shingles.select("filename", "minhash").collect()
similar_pairs_minhash = []

for r1, r2 in combinations(rows, 2):
    sim = CompareSignatures.compare(r1["minhash"], r2["minhash"])
    if sim >= threshold_minhash:
        similar_pairs_minhash.append((r1["filename"], r2["filename"], sim))

print("Pairs with MinHash similarity >= 0.6:")
for f1, f2, sim in similar_pairs_minhash:
    print(f"{f1} ~ {f2} : {sim:.2f}")


Pairs with MinHash similarity >= 0.6:


**Test à part**

In [62]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col
# import mmh3
# from itertools import combinations

# # --- Spark session ---
# spark = SparkSession.builder.master("local[*]").appName("Test MinHash").getOrCreate()

# # --- Exemples de textes ---
# docs = [
#     ("doc1.txt", "The quick brown fox jumps over the lazy dog."),
#     ("doc2.txt", "The quick brown fox jumps over the lazy dog!"),  # très similaire à doc1
#     ("doc3.txt", "Alice was beginning to get very tired of sitting by her sister on the bank."),
#     ("doc4.txt", "It was the best of times, it was the worst of times."),
#     ("doc5.txt", "Mr. Utterson, the lawyer, was a man of a rugged countenance.")
# ]

# # --- Création du DataFrame Spark ---
# df_docs = spark.createDataFrame(docs, ["filename", "text"])

# # --- Shingling (comme ta classe Shingling) ---
# shingler = Shingling(k=3)  # 3 mots par shingle pour exemple
# df_shingles = shingler.transform(df_docs, "text")

# # --- MinHashing ---
# n_hashes = 50
# minhasher = MinHashing(n_hashes)
# df_shingles = df_shingles.withColumn("minhash", minhasher.minhash_udf()(col("hashed_shingles")))

# # --- Comparaison MinHash ---
# threshold_minhash = 0.6
# rows = df_shingles.select("filename", "minhash").collect()
# similar_pairs_minhash = []

# for r1, r2 in combinations(rows, 2):
#     sim = CompareSignatures.compare(r1["minhash"], r2["minhash"])
#     if sim >= threshold_minhash:
#         similar_pairs_minhash.append((r1["filename"], r2["filename"], sim))

# print("Pairs with MinHash similarity >= 0.6:")
# for f1, f2, sim in similar_pairs_minhash:
#     print(f"{f1} ~ {f2} : {sim:.2f}")


Pairs with MinHash similarity >= 0.6:
doc1.txt ~ doc2.txt : 1.00
